In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as Data
from tqdm import tqdm

import numpy as np

In [2]:
USE_CUDA = True

In [3]:
root = '/notebooks/sinica/dataset/'
train_data = root+'facial.train'
dev_data = root+'facial.dev'
test_data = root+'facial.test'

UNKOWN_TAG = "<UNKNOWN>"
PAD_TAG = "<PAD>"
tag_to_ix = {UNKOWN_TAG: 0, PAD_TAG:1, "B-Func": 2, "I-Func": 3, "O": 4}

tagset_size = len(tag_to_ix)
MAX_LEN = 100
BATCH_SIZE = 128

EMBEDDING_DIM = 20
HIDDEN_DIM1 = 10
HIDDEN_DIM2 = 8
LABEL_EMBED_DIM = 3
DENSE_OUT = 100

ATTN_IN = tagset_size+LABEL_EMBED_DIM
ATTN_OUT = 5

In [4]:
def readfile(data):
    with open(data, "r", encoding="utf-8") as f:
        content = f.read().splitlines()
        
    return content

# ==================================================

def get_word_and_label(_content, start_w, end_w):
    word_list = []
    tag_list = []
    for word_set in _content[start_w:end_w]:
        word_list.append(word_set[0])
        tag_list.append(word_set[2:])
    
    return word_list, tag_list

def split_to_list(content):
    init = 0
    word_list = []
    tag_list = []

    for i, c in enumerate(content):
        if c=='':
            words, tags = get_word_and_label(content, init, i)
            init = i+1
            word_list.append(words)
            tag_list.append(tags)
            
    return word_list, tag_list

# ==================================================
    
def prepare_sequence(seq, to_ix):
    idxs = []
    for w in seq:
        if w not in to_ix:
            idxs.append(to_ix[UNKOWN_TAG])
        else:
            idxs.append(to_ix[w])
    
#     idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def prepare_all(seqs, to_ix):
    seq_list = []
    for i in range(len(seqs)):
        seq_list.append(prepare_sequence(seqs[i], to_ix))
        
    seq_list = torch.stack(seq_list)
        
    return seq_list

# ==================================================

def word2index(word_list):
    word_to_ix = {"<UNKNOWN>":0, "<PAD>":1}
    for sentence in word_list:
        for word in sentence:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
                
    return word_to_ix

def dict_inverse(tag_to_ix):
    ix_to_tag = {v: k for k, v in tag_to_ix.items()}
    return ix_to_tag

def index2tag(indexs, ix_to):
    to_tags = [ix_to[i] for i in indexs.cpu().numpy()]
    return to_tags

# ==================================================

def find_max_len(word_list):
    max_len = 0
    for i in range(len(word_list)):
        if max_len<len(word_list[i]):
            max_len=len(word_list[i])
            
    return max_len

# ====== filter the length of sentence more than MAX_LEN =======

def filter_len(word_list):
    reserved_index = []
    for i in range(len(word_list)):
        if len(word_list[i])<MAX_LEN:
            reserved_index.append(i)
            
    return reserved_index


def filter_sentence(reserved_index, word_list, tag_list):
    filter_word = list(word_list[i] for i in reserved_index)
    filter_tag = list(tag_list[i] for i in reserved_index)
    return filter_word, filter_tag

# ==================================================

def pad_seq(seq):
    seq += [PAD_TAG for i in range(MAX_LEN-len(seq))]
    return seq

def pad_all(filter_word, filter_tag):
    input_padded = [pad_seq(s) for s in filter_word]
    target_padded = [pad_seq(s) for s in filter_tag]
    
    return input_padded, target_padded

# ==================================================
def dataload(input_var, target_var):
    torch_dataset = Data.TensorDataset(input_var, target_var)

    loader = Data.DataLoader(
        dataset=torch_dataset,      # torch TensorDataset format
        batch_size=BATCH_SIZE,      # mini batch size
        shuffle=True,               
        num_workers=2,       
        drop_last=True
    )
    
    return loader

# ==================================================
def softmax_output(output):
    output = output.view(BATCH_SIZE,tagset_size).argmax(1)
    return output

In [34]:
class Attn(nn.Module):
    def __init__(self, attn_input, attn_output):
        super(Attn, self).__init__()
        
        self.attn_input = attn_input
        self.attn_output = attn_output
        
        self.w1 = nn.Linear(self.attn_input, self.attn_output)
        self.w2 = nn.Linear(self.attn_input, self.attn_output)
        self.tanh = nn.Tanh()
        self.v = nn.Parameter(torch.FloatTensor(128,1,attn_output))
        
        
    def forward(self, encoder_outputs):
#         this_batch_size = encoder_outputs.size(0)
#         max_len = encoder_outputs.size(1)
        
        decoder = encoder_outputs[:,-1,:].unsqueeze(1)                       #B*1*(ts+LE) [128,1,8]
        print(decoder.size())
        
        encoder_score = self.w1(encoder_outputs)
        decoder_score = self.w2(decoder)
        energy = self.tanh(encoder_score+decoder_score)
        print(energy.size())
        
        energy = torch.bmm(self.v, energy)
#         energy = self.v.bmm(energy)
        
        return F.softmax(energy)
    
    
    

#     def forward111111(self, encoder_outputs):
#         this_batch_size = encoder_outputs.size(0)
#         max_len = encoder_outputs.size(1)
        
# #         decoder_seq = encoder_outputs

#         # Create variable to store attention energies
#         attn_energies = Variable(torch.zeros(this_batch_size, max_len)) # B x L

#         if USE_CUDA:
#             attn_energies = attn_energies.cuda()

#         # For each batch of encoder outputs
#         for b in range(this_batch_size):
#             # Calculate energy for each encoder output
#             for i in range(max_len):
#                 decoder_seq = encoder_outputs[b, i]
#                 attn_energies[b, i] = self.score(decoder_seq, encoder_outputs[b, 0:i].unsqueeze(0))

#         # Normalize energies to weights in range 0 to 1, resize to 1 x B x S
#         return F.softmax(attn_energies).unsqueeze(1)
    
#     def score(self, hidden, encoder_output):

# #         energy = self.attn(torch.cat((hidden, encoder_output), 1))
#         e1 = self.w1(encoder_output)
#         e2 = self.w2(hidden)
#         energy = self.tanh(e1+e2)
#         energy = self.v.dot(energy)
        
#         return energy

In [16]:
class Entity_Typing(nn.Module):
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim1, hidden_dim2, \
                 label_embed_dim):
        super(Entity_Typing, self).__init__()
        self.embedding_dim = embedding_dim                   #E
        self.hidden_dim1 = hidden_dim1                       #h1
        self.hidden_dim2 = hidden_dim2                       #h2
        self.label_embed_dim = label_embed_dim               #LE
        self.vocab_size = vocab_size                         #vs
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)                    #ts
        
        
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.bilstm = nn.LSTM(embedding_dim, hidden_dim1 // 2,
                            num_layers=2, bidirectional=True, batch_first=True)
        
        self.dense = nn.Linear(hidden_dim1, DENSE_OUT)
        
        self.top_hidden = nn.LSTMCell(DENSE_OUT+label_embed_dim, hidden_dim2)          
        

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim2, self.tagset_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.label_embed = nn.Linear(self.tagset_size, self.label_embed_dim)
        
        
        self.attn = Attn(ATTN_IN, ATTN_OUT)
        
#         self.hidden1 = self.init_hidden1()
#         self.hidden2 = self.init_hidden2()
#         self.to_label_embed = self.init_label_embed()

# 在init宣告變數不會覆蓋，如果要存下來才要宣告在init
        
        
    def init_hidden1(self):       
        hidden = torch.randn(2*2, BATCH_SIZE, self.hidden_dim1 // 2)    #4*B*(h1/2)
#         hidden = Variable(hidden.data, requires_grad=True)

        return (hidden.cuda(), hidden.cuda())if USE_CUDA else (hidden,hidden)
    
    def init_hidden2(self):       
        hidden = torch.randn(BATCH_SIZE, self.hidden_dim2)              #B*h2
#         hidden = Variable(hidden.data, requires_grad=True)

        return (hidden.cuda(), hidden.cuda())if USE_CUDA else (hidden,hidden)
    
    def init_label_embed(self):
        hidden = torch.zeros(BATCH_SIZE, self.label_embed_dim)          #B*LE
        return hidden.cuda()if USE_CUDA else hidden
    
    def create_output(self):
        output_tensor = torch.zeros(BATCH_SIZE, MAX_LEN, self.tagset_size)  #B*ML*ts
        return output_tensor.cuda()if USE_CUDA else output_tensor
        
    def forward(self, sentence):
        self.hidden1 = self.init_hidden1()                      #4*B*(h1/2)
        output_tensor = self.create_output()                    #B*ML*ts
        
        embeds = self.word_embeds(sentence)                     #B*ML*E,[128, 100, 20]
        
        bilstm_out, self.hidden1 = self.bilstm(embeds, self.hidden1)
        # bilstm_out -> B*ML*h1,[128, 100, 10]
        # self.hidden1 -> ( 4*B*(h1/2), 4*B*(h1/2) )
        
        dense_out = self.dense(bilstm_out)                      #B*ML*DENSE_OUT,[128, 100, 100]
        
        
        encoder_sequence_l = []
        decoder_sequence = [] 
        

        for length in range(MAX_LEN):
            now_token = dense_out[:,length,:]
            now_token = torch.squeeze(now_token, 1)
            if length==0:
                
#                 fake_hidden=(100)
#                 noise_x = random(100)
                self.hidden2 = self.init_hidden2()
                self.zero_label_embed = self.init_label_embed()
                combine_x = torch.cat((now_token, self.zero_label_embed),1)  #B*(DENSE_OUT+LE),[128, 103]
                
            else:
#                 fake_hidden=h
                self.hidden2 = (h_next, c_next)
                combine_x = torch.cat((now_token, label),1)

            h_next, c_next = self.top_hidden(combine_x, self.hidden2)    #B*h2,[128, 8]           
            to_tags = self.hidden2tag(h_next)                            #B*ts,[128, 5]            
            output = self.softmax(to_tags)                               #B*ts,[128, 5]             
            label = self.label_embed(output)                             #B*LE,[128, 3]
            
            # to the index
            s_output = softmax_output(output)
            
#             print(s_output)
            
            for i, tag in enumerate(s_output):
                if tag==4:
                    to_tags[i] = torch.FloatTensor([-999999 * self.tagset_size])
                    label[i] = torch.FloatTensor([-999999 * self.tagset_size])
                    
            
            # relation layer
            encoder_sequence_l.append(torch.cat((to_tags,label),1))
            encoder_sequence = torch.stack(encoder_sequence_l).t()
#             print(encoder_sequence)
            
            
#             print(encoder_sequence.size())                              #B*len*(ts+LE), [128,1,8]
            # Calculate attention weights 
            attn_weights = self.attn(encoder_sequence)
            
        
        
            # entity output
            output_tensor[:,length,:] = output
        
        
        
        
        
        
        '''NLLLoss input: Input: (N,C) where C = number of classes'''
        return output_tensor.view(BATCH_SIZE*MAX_LEN, self.tagset_size), encoder_sequence

In [7]:
class Relation_Extraction(nn.Module):
    def __init__(self, ):
        super(Relation_Extraction, self).__init__()
        pass
        
    def forward(self):
        pass

In [8]:
ix_to_tag = dict_inverse(tag_to_ix)
#===============================================
content = readfile(train_data)
word_list, tag_list = split_to_list(content)
word_to_ix = word2index(word_list)
reserved_index = filter_len(word_list)
filter_word, filter_tag = filter_sentence(reserved_index, word_list, tag_list)
input_padded, target_padded = pad_all(filter_word, filter_tag)
#================================================
input_var = prepare_all(input_padded, word_to_ix)
target_var = prepare_all(target_padded, tag_to_ix)
#================================================
vocab_size = len(word_to_ix)

In [35]:
loader = dataload(input_var, target_var)
entity_typing = Entity_Typing(vocab_size, tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM1, HIDDEN_DIM2, \
              LABEL_EMBED_DIM).cuda()
optimizer = optim.Adam(entity_typing.parameters(), lr=0.01, weight_decay=1e-4)
criterion = nn.NLLLoss()

In [10]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [36]:
n_iters = 10
print_every = 12
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for epoch in tqdm(range(10)):  
    for step, (batch_x, batch_y) in enumerate(loader):
        optimizer.zero_grad()
        output, ee = entity_typing(batch_x.cuda() if USE_CUDA else batch_x)
        batch_y = batch_y.view(BATCH_SIZE*MAX_LEN)
        loss = criterion(output, batch_y.cuda() if USE_CUDA else batch_y)
        loss.backward()
#         loss.backward(retain_graph=True)
        optimizer.step()
        break

        
        if step % print_every == 1:
            all_losses.append(loss.cpu())
        #    print('%.4f| epoch: %d| step: %d| %s' % (loss, epoch, step, timeSince(start)))
    print("epoch: %d | loss %.4f" % (epoch,loss))
    
    break

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([128, 1, 8])
torch.Size([128, 1, 5])


RuntimeError: invalid argument 6: wrong matrix size at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:477

In [ ]:
ee.size()

In [ ]:
len(all_losses)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.plot(all_losses[:100])
plt.xlabel("Step")
plt.ylabel("Loss") 
plt.show()

In [ ]:
# for one input
def easy_pad(easy_sent, easy_tar):
    easy_sent += [PAD_TAG for i in range(MAX_LEN-len(easy_sent))]
    easy_tar += [PAD_TAG for i in range(MAX_LEN-len(easy_tar))]
    
    return easy_sent, easy_tar

def easy_test(_input):
    _input = torch.unsqueeze(_input, 0).expand(128,100)
    return _input

def easy_output(output):
    output = output.view(128,100,5)[0].argmax(1)
    return output

In [ ]:
test_content = readfile(test_data)
word_list_test, tag_list_test = split_to_list(test_content)

#===================================
easy_sent, easy_tar = easy_pad(word_list_test[3],tag_list_test[3])
input_test = prepare_sequence(easy_sent, word_to_ix)
# input_test = prepare_sequence(t_sentence, word_to_ix)

target_test = prepare_sequence(easy_tar, tag_to_ix)

_input = easy_test(input_test)

In [ ]:
# Check predictions after training
with torch.no_grad():
    output = entity_typing(_input.cuda() if USE_CUDA else _input)
    output = easy_output(output)
    
    print('predict :', output)
    print('true :', target_test)
    

In [ ]:
def prepare_sequence__(seq, to_ix):
    gg = []
    
    for w in seq:
        if w not in to_ix:
            return seq

#     idxs = [to_ix[w] for w in seq]
#     return torch.tensor(idxs, dtype=torch.long)

def prepare_all__(seqs, to_ix):
    get_index = []
    notinseq = []
    for i in range(len(seqs)):
        a = prepare_sequence__(seqs[i], to_ix)
        if a != None:
            notinseq.append( a)
            get_index.append(i)
#         seq_list.append(prepare_sequence__(seqs[i], to_ix))
        
#     notinseq = torch.stack(notinseq)
        
    return notinseq, get_index



In [ ]:
# for batch input
reserved_index_test = filter_len(word_list_test[:143])

In [ ]:
len(reserved_index_test)

In [ ]:
filter_word, filter_tag = filter_sentence(reserved_index_test, word_list_test, tag_list_test)
input_padded, target_padded = pad_all(filter_word, filter_tag)
input_var = prepare_all(input_padded, word_to_ix)
target_var = prepare_all(target_padded, tag_to_ix)

In [ ]:
def total_output(output):
    output = output.view(BATCH_SIZE,MAX_LEN,tagset_size).argmax(2)
    return output

In [ ]:
# Check predictions after training
with torch.no_grad():
    output = entity_typing(input_var.cuda() if USE_CUDA else _input)
    
    loss = criterion(output.cpu(), target_var.view(128*100))
    output = total_output(output)
    
    print('predict :', output[37])
    print('true :', target_var[37])
    print()
    print('predict :', index2tag(output[37], ix_to_tag))
    print('true :', index2tag(target_var[37], ix_to_tag))
    print()
    
    print("Loss : %.4f" % loss)

In [ ]:
Variable(torch.Tensor([0] * 12))

In [ ]:
encoder_sequence = Variable(torch.zeros([10, 1]))
encoder_sequence[1].cat(3)

In [ ]:
torch.FloatTensor(1, 5)